# Effective Pandas Patterns for Data Manipulation

In [1]:
import pandas as pd
import numpy as np

## 5. Series Deep Dive

In [2]:
url = 'http://github.com/mattharrison/datasets/raw/master/data/' \
    'vehicles.csv.zip'

df = pd.read_csv(url)
city_mpg = df.city08
highway_mpg = df.highway08

C:\Users\rober\AppData\Local\Temp\ipykernel_18524\548609446.py:4: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [2]:
# Attributes on a series:
len(dir(city_mpg))

420

Category of these methods:
- Dunder methods: (.__add__, .__iter__, etc.)
- Corresponding operator methods (.add -> .__add__)
- Aggregate methods (.mean, .median, etc.)
- Conversion methods (.to_)
- Manipulation methods (.sort_values, .drop, etc.)
- Indexing and accessor methods (.loc, .iloc, etc.)
- String (.str.), date (.dt.), plotting (.plot.), categorical (.cat.) manipulation
- Transformation methods (.unstack, .reset_index)
- Attributes (.index, .dtypes)
- Private attributes

## 6. Operators (and Dunder Methods)

In [3]:
# Dunder methods
print(2 + 4)
(2).__add__(4)

6


6

To perform operations between pandas Series, you will want to make sure that the indexes:
- are unique (no duplicates)
- are common to both series

Iterations: avoid using a for loop with a series

In [6]:
# Operator methods: we can use the fill_valueto specify that we
# use another value instead

s1 = pd.Series([10, 20, 30], index=[1, 2, 2])
s2 = pd.Series([35, 44, 53], index=[2, 2, 4], name="s2")

print(s1 + s2)
s1.add(s2, fill_value=0)

1     NaN
2    55.0
2    64.0
2    65.0
2    74.0
4     NaN
dtype: float64


1    10.0
2    55.0
2    64.0
2    65.0
2    74.0
4    53.0
dtype: float64

In [8]:
# Chaining: it is preferable to use the method instead of the opeator
# because it makes chaining manipulation easier.

print((city_mpg + highway_mpg) / 2)

# Using methods:
(city_mpg
.add(highway_mpg)
.div(2)
)

0        22.0
1        11.5
2        28.0
3        11.0
4        20.0
         ... 
41139    22.5
41140    24.0
41141    21.0
41142    21.0
41143    18.5
Length: 41144, dtype: float64


0        22.0
1        11.5
2        28.0
3        11.0
4        20.0
         ... 
41139    22.5
41140    24.0
41141    21.0
41142    21.0
41143    18.5
Length: 41144, dtype: float64

## 7. Aggregate methods

Aggregations allow you to take detailed data and collapse it to a single value.

In [10]:
# Count and percentage
(city_mpg
.gt(20)
.sum()
)

(city_mpg
.gt(20)
.mul(100)
.mean()
)

24.965973167412017

In [11]:
# .agg and aggregation strings

city_mpg.agg(['mean', max, 'skew'])

mean     18.369045
max     150.000000
skew      7.705198
Name: city08, dtype: float64

In [12]:
(highway_mpg
.isna()
.sum()
)

0

In [20]:
def count_not_na(s):
    return s.notna().sum()
highway_mpg.agg([count_not_na, 'size', 'unique', 'mean', 'max'])

count_not_na                                                41144
size                                                        41144
unique          [25, 14, 33, 12, 23, 24, 29, 26, 31, 30, 15, 2...
mean                                                    24.504667
max                                                           124
Name: highway08, dtype: object

## 8. Conversion methods

Using the correct type can save significant amounts of memory

In [27]:
np.iinfo('int16')

# Memory usage:
city_mpg.nbytes
city_mpg.memory_usage()

329280

A categorical series is useful for string data and can result in large memory savings.

## 9. Manipulation methods

In [37]:
# .apply and .where

mask = df.make.value_counts().index[:5]
np.where(df.make.isin(mask), df.make, 'Other')


array(['Other', 'Other', 'Dodge', ..., 'Other', 'Other', 'Other'],
      dtype=object)

In [50]:
# Filling In Missing Data

# - Drop na (.dropna())
# - ffill (.ffill())
# - interpolate (.interpolate())
# - fillna (with mean, median, etc)

# Clip allows you to remove values that might be outliers
print(len(city_mpg))
(city_mpg
.clip(lower=city_mpg.quantile(.05),
      upper=city_mpg.quantile(.95))
.index
)
#city_mpg[city_mpg < 11]

41144


RangeIndex(start=0, stop=41144, step=1)

In [7]:
# Ranking data

(city_mpg
.rank(method='dense')
)
# help(pd.Series.rank)

0        14.0
1         4.0
2        18.0
3         5.0
4        12.0
         ... 
41139    14.0
41140    15.0
41141    13.0
41142    13.0
41143    11.0
Name: city08, Length: 41144, dtype: float64

In [9]:
pd.cut(city_mpg, 10)

0        (5.856, 20.4]
1        (5.856, 20.4]
2         (20.4, 34.8]
3        (5.856, 20.4]
4        (5.856, 20.4]
             ...      
41139    (5.856, 20.4]
41140    (5.856, 20.4]
41141    (5.856, 20.4]
41142    (5.856, 20.4]
41143    (5.856, 20.4]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64, right]): [(5.856, 20.4] < (20.4, 34.8] < (34.8, 49.2] < (49.2, 63.6] ... (92.4, 106.8] < (106.8, 121.2] < (121.2, 135.6] < (135.6, 150.0]]

In [26]:
# Exercises:
# 1
np.where(city_mpg >= city_mpg.mean(), 'high', 'low')

mean_city_mpg = np.mean(city_mpg)
def compare(e):
    if e >= mean_city_mpg:
        return 'high'
    return 'low'

%timeit -n 100 city_mpg.apply(compare)

# 2
%timeit -n 100 np.select([city_mpg >= city_mpg.mean(), city_mpg < city_mpg.mean()], ['high', 'low'])
%timeit -n 100 np.where(city_mpg >= city_mpg.mean(), 'high', 'low')



5.83 ms ± 665 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
988 µs ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
357 µs ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
